In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT
from probs_saver import ProbStore

datapath = '../input/predict_probs_resnet50.h5'
save_to = '../input/resnet_test_comp.h5'

In [2]:
%%time
store = pd.HDFStore(datapath)
keys = store.keys()
print('{} tables in storage'.format(len(keys)))

12091 tables in storage
CPU times: user 47.4 s, sys: 1.25 s, total: 48.6 s
Wall time: 1min


In [3]:
cols = []
for i in range(1, 5271):
    cols.append(str(i))
    
saver = ProbStore(path=save_to)

for key in tqdm(keys): 
    table = store.select(key)
    assert table.shape[0] % 10 == 0
    two_cols = []
    means_df = pd.DataFrame(columns=cols, dtype=np.float16)
    for i in range(int(table.shape[0] / 10)):
        st =  10 * i
        product_id = table.pr_id[st]
        img_num = table.img_num[st]
        sum1 = table.iloc[pd.np.r_[st : st + 4, st + 5:st + 9]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        sum2 = table.iloc[pd.np.r_[st + 4: st + 5, st + 9:st + 10]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        mean_ = (sum1 + 3*sum2) / 14.
        #print(sum_)
        
        two_cols.append((product_id, img_num))
        means_df = means_df.append(mean_, ignore_index=True)
        #print(sums_df)
    two_cols_df = pd.DataFrame(data=two_cols, columns=['pr_id', 'img_num'], dtype=np.int32)
    df = pd.concat([two_cols_df, means_df], axis=1)
    #print(df)
    saver.saveProbs(df)
    # debug
   # print(df)
    #break

100%|██████████| 12091/12091 [7:02:11<00:00,  2.10s/it]  
